In [1]:
from mongo_processor.NeuroscienceNewsMongoConnector import NeuroscienceNewsMongoConnector

In [2]:
from neuroscience_news.NeuroscienceNewsScraper import NeuroscienceNewsArticleScraper

In [3]:
from joblib import Parallel, delayed

In [4]:
articles_file = "./data/neuroscience_news_urls.txt"

In [5]:
with open(articles_file) as file:
    articles = file.read().splitlines()

In [6]:
db_name = "Neuroscience_News"
sc = NeuroscienceNewsArticleScraper()
mc = NeuroscienceNewsMongoConnector(db_name)

In [7]:
scraped_articles = Parallel(n_jobs=-1)(delayed(sc.scrape_article)(article) for article in articles)

SSLError: None: Max retries exceeded with url: /coffee-stimulants-high-achievers-slack-off/?User-Agent=Mozilla%2F5.0+%28X11%3B+Ubuntu%3B+Linux+x86_64%3B+rv%3A52.0%29+Gecko%2F20100101+Firefox%2F52.0&Accept-Encoding=gzip%2C+deflate&Accept=%2A%2F%2A&Connection=keep-alive (Caused by None)

In [ ]:
mc.save_neuroscience_articles(scraped_articles)